In [ ]:
import os
import xml.etree.ElementTree as ET
import csv
from collections import defaultdict

def count_egram_by_roleset(input_directory, output_csv_file):
    """
    Quét một thư mục chứa các tệp XML, đếm các mẫu có src="EGRAM"
    cho mỗi ROLESET ID (ví dụ: abolish.01, abolish.02) và ghi kết quả vào tệp CSV.
    """
    # Sử dụng defaultdict để tự động khởi tạo số đếm
    roleset_counts = defaultdict(int)

    print(f"Bắt đầu quét thư mục: '{input_directory}'...")

    if not os.path.isdir(input_directory):
        print(f"Lỗi: Thư mục '{input_directory}' không tồn tại.")
        return

    for filename in os.listdir(input_directory):
        if filename.endswith('.xml'):
            file_path = os.path.join(input_directory, filename)
            print(f"Đang xử lý tệp: {filename}")

            try:
                tree = ET.parse(file_path)
                root = tree.getroot()

                # Lặp qua từng roleset thay vì predicate ###
                # Chúng ta sẽ đi sâu hơn vào cấu trúc XML
                for predicate in root.findall('.//predicate'):
                    # Bên trong mỗi predicate, tìm tất cả các roleset
                    for roleset in predicate.findall('roleset'):
                        roleset_id = roleset.get('id')
                        if roleset_id:
                            # Tìm các example chỉ thuộc roleset này
                            egram_examples = roleset.findall("example[@src='EGRAM']")
                            count = len(egram_examples)

                            if count > 0:
                                # Dùng roleset_id làm khóa đếm ###
                                roleset_counts[roleset_id] += count

            except ET.ParseError as e:
                print(f"Lỗi phân tích cú pháp tệp {filename}: {e}")
            except Exception as e:
                print(f"Đã xảy ra lỗi không xác định với tệp {filename}: {e}")

    print(f"\nĐang ghi kết quả vào tệp: '{output_csv_file}'...")
    try:
        with open(output_csv_file, 'w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)

            # Cập nhật tiêu đề cột trong file CSV ###
            csv_writer.writerow(['Roleset_ID', 'Egram_Count'])

            # Ghi dữ liệu, sắp xếp theo roleset ID
            for roleset_id, count in sorted(roleset_counts.items()):
                csv_writer.writerow([roleset_id, count])

        print("Hoàn thành! Tệp CSV đã được tạo thành công.")

    except IOError as e:
        print(f"Lỗi: Không thể ghi vào tệp {output_csv_file}. Vui lòng kiểm tra quyền truy cập. Lỗi: {e}")


# --- CẤU HÌNH ---
if __name__ == "__main__":
    input_folder = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/GramVar'
    output_file = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/thong_ke_egram_by_sense.csv'

    if not os.path.exists(input_folder):
        os.makedirs(input_folder)
        print(f"Đã tạo thư mục mẫu '{input_folder}'.\n"
              f"Vui lòng sao chép các tệp XML của bạn vào đó và chạy lại chương trình.")
    else:
        # Gọi hàm đã cập nhật
        count_egram_by_roleset(input_folder, output_file)

Bắt đầu quét thư mục: '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/GramVar'...
  -> Đang xử lý tệp: begin_2_full.xml
  -> Đang xử lý tệp: begin_1_full.xml
  -> Đang xử lý tệp: translate_2_full.xml
  -> Đang xử lý tệp: transform_1_full.xml
  -> Đang xử lý tệp: delete_full.xml
  -> Đang xử lý tệp: confer_full.xml
  -> Đang xử lý tệp: splice_2_full.xml
  -> Đang xử lý tệp: translate_1_full.xml
  -> Đang xử lý tệp: inhibit_full.xml
  -> Đang xử lý tệp: proliferate_full.xml
  -> Đang xử lý tệp: result_full.xml
  -> Đang xử lý tệp: modify_full.xml
  -> Đang xử lý tệp: lead_full.xml
  -> Đang xử lý tệp: decrease_2_full.xml
  -> Đang xử lý tệp: splice_1_full.xml
  -> Đang xử lý tệp: lose_full.xml
  -> Đang xử lý tệp: transform_2_full.xml
  -> Đang xử lý tệp: eliminate_full.xml
  -> Đang xử lý tệp: encode_full.xml
  -> Đang xử lý tệp: block_full.xml
  -> Đang xử lý tệp: develop_full.xml
  -> Đang xử lý tệp: decrease_1_full.xml
  -> Đang xử lý tệp: recognize_

In [ ]:
import os
import xml.etree.ElementTree as ET
import csv
from collections import defaultdict

def count_variations_by_roleset(input_directory, output_csv_file):
    """
    Quét một thư mục chứa các tệp XML, đếm các mẫu có src="PERMUTATE" và src="REPLACE"
    cho mỗi ROLESET ID, tính tổng VE và ghi kết quả vào tệp CSV.
    """
    # Cấu trúc dữ liệu để lưu trữ số đếm:
    # { 'roleset_id': {'per': count, 'rep': count} }
    roleset_counts = defaultdict(lambda: {"per": 0, "rep": 0})

    print(f"Bắt đầu quét thư mục: '{input_directory}'...")

    if not os.path.isdir(input_directory):
        print(f"Lỗi: Thư mục '{input_directory}' không tồn tại.")
        return

    for filename in os.listdir(input_directory):
        if filename.endswith('.xml'):
            file_path = os.path.join(input_directory, filename)
            print(f"Đang xử lý tệp: {filename}")

            try:
                tree = ET.parse(file_path)
                root = tree.getroot()

                for predicate in root.findall('.//predicate'):
                    # Bên trong mỗi predicate, tìm tất cả các roleset
                    for roleset in predicate.findall('roleset'):
                        # Lấy roleset id và chuyển sang chữ thường
                        roleset_id = roleset.get('id').lower()
                        if roleset_id:
                            # Tìm các example chỉ thuộc roleset này
                            examples = roleset.findall("example")
                            for ex in examples:
                                src_value = ex.get('src')

                                if src_value == 'PERMUTATE':
                                    # Dùng roleset_id làm khóa
                                    roleset_counts[roleset_id]['per'] += 1
                                elif src_value == 'REPLACE':
                                    roleset_counts[roleset_id]['rep'] += 1

            except ET.ParseError as e:
                print(f"Lỗi phân tích cú pháp tệp {filename}: {e}")
            except Exception as e:
                print(f"Đã xảy ra lỗi không xác định với tệp {filename}: {e}")

    print(f"\nĐang ghi kết quả vào tệp: '{output_csv_file}'...")
    try:
        with open(output_csv_file, 'w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)

            # Cập nhật tiêu đề cột
            csv_writer.writerow(['Roleset_ID', 'Permute_Count (per)', 'Replace_Count (rep)', 'Total_VE'])

            for roleset_id, counts in sorted(roleset_counts.items()):
                per_count = counts['per']
                rep_count = counts['rep']
                total_ve = per_count + rep_count
                csv_writer.writerow([roleset_id, per_count, rep_count, total_ve])

        print("Hoàn thành! Tệp CSV đã được tạo thành công.")

    except IOError as e:
        print(f"Lỗi: Không thể ghi vào tệp {output_csv_file}. Vui lòng kiểm tra quyền truy cập. Lỗi: {e}")

# --- CẤU HÌNH ---
if __name__ == "__main__":
    input_folder = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/ParaVE'
    output_file = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/thong_ke_variations_by_sense.csv'

    if not os.path.exists(input_folder):
        os.makedirs(input_folder)
        print(f"Đã tạo thư mục mẫu '{input_folder}'.\n"
              f"Vui lòng sao chép các tệp XML của bạn vào đó và chạy lại chương trình.")
    else:
        # Gọi hàm đã cập nhật
        count_variations_by_roleset(input_folder, output_file)

Bắt đầu quét thư mục: '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/Clean_Dataset/Corpus/ParaVE'...
  -> Đang xử lý tệp: catalyse_full.xml
  -> Đang xử lý tệp: result_full.xml
  -> Đang xử lý tệp: develop_full.xml
  -> Đang xử lý tệp: eliminate_full.xml
  -> Đang xử lý tệp: translate_3_full.xml
  -> Đang xử lý tệp: transform_1_full.xml
  -> Đang xử lý tệp: begin_2_full.xml
  -> Đang xử lý tệp: block_full.xml
  -> Đang xử lý tệp: abolish_full.xml
  -> Đang xử lý tệp: translate_2_full.xml
  -> Đang xử lý tệp: inhibit_full.xml
  -> Đang xử lý tệp: begin_1_full.xml
  -> Đang xử lý tệp: splice_2_full.xml
  -> Đang xử lý tệp: modify_full.xml
  -> Đang xử lý tệp: lose_full.xml
  -> Đang xử lý tệp: lead_full.xml
  -> Đang xử lý tệp: encode_full.xml
  -> Đang xử lý tệp: proliferate_full.xml
  -> Đang xử lý tệp: express_full.xml
  -> Đang xử lý tệp: alter_full.xml
  -> Đang xử lý tệp: decrease_1_full.xml
  -> Đang xử lý tệp: recognize_full.xml
  -> Đang xử lý tệp: translate_1_full

In [ ]:
import pandas as pd
import os

def merge_and_format_stats_files(egram_file, variations_file, output_file):
    """
    Đọc hai tệp CSV thống kê, hợp nhất, định dạng, sắp xếp,
    thêm dòng tổng cộng và lưu kết quả.
    """
    print(f"Đang đọc tệp: '{egram_file}'")
    egram_df = pd.read_csv(egram_file)
    print(f"Đang đọc tệp: '{variations_file}'")
    variations_df = pd.read_csv(variations_file)
    print("Đang hợp nhất hai tệp thống kê...")
    merged_df = pd.merge(egram_df, variations_df, on='Roleset_ID', how='outer')
    merged_df = merged_df.fillna(0)
    count_columns = ['Egram_Count', 'Permute_Count (per)', 'Replace_Count (rep)', 'Total_VE']
    for col in count_columns:
        if col in merged_df.columns:
            merged_df[col] = merged_df[col].astype(int)

    column_rename_map = {
        'Roleset_ID': 'predicate/type',
        'Egram_Count': 'egram',
        'Total_VE': 'VE',
        'Permute_Count (per)': 'per',
        'Replace_Count (rep)': 'rep'
    }
    merged_df = merged_df.rename(columns=column_rename_map)

    new_column_order = [
        'predicate/type',
        'egram',
        'VE',
        'per',
        'rep'
    ]
    merged_df = merged_df[new_column_order]
    merged_df['SUM'] = merged_df['egram'] + merged_df['VE']

    # Logic sắp xếp
    temp_df = merged_df['predicate/type'].str.rsplit('.', n=1, expand=True)
    merged_df['lemma_sort_key'] = temp_df[0].str.strip()
    merged_df['sense_sort_key'] = pd.to_numeric(temp_df[1])
    merged_df = merged_df.sort_values(by=['lemma_sort_key', 'sense_sort_key'], ascending=True)
    merged_df = merged_df.drop(columns=['lemma_sort_key', 'sense_sort_key'])

    # Tính tổng của tất cả các cột là số
    column_sums = merged_df.select_dtypes(include='number').sum()

    # Tạo một dòng mới cho bảng tổng kết
    total_row = pd.DataFrame([column_sums])

    # Nối dòng tổng kết vào cuối DataFrame gốc
    merged_df = pd.concat([merged_df, total_row], ignore_index=True)

    # Lưu DataFrame đã xử lý vào tệp CSV mới
    try:
        # Chuyển các cột số về dạng số nguyên (vì concat có thể đổi chúng thành float)
        final_numeric_cols = ['egram', 'VE', 'per', 'rep', 'SUM']
        for col in final_numeric_cols:
             merged_df[col] = merged_df[col].astype(int)

        merged_df.to_csv(output_file, index=False, encoding='utf-8')
        print(f"\nThành công! Đã tạo tệp tổng hợp đã định dạng tại: '{output_file}'")
    except Exception as e:
        print(f"Lỗi khi ghi tệp: {e}")


# --- CẤU HÌNH ---
if __name__ == "__main__":
    base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/'

    egram_input = os.path.join(base_path, 'thong_ke_egram_by_sense.csv')
    variations_input = os.path.join(base_path, 'thong_ke_variations_by_sense.csv')

    sencount_output = os.path.join(base_path, 'sencount.csv')

    merge_and_format_stats_files(egram_input, variations_input, sencount_output)

Đang đọc tệp: '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/thong_ke_egram_by_sense.csv'
Đang đọc tệp: '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/thong_ke_variations_by_sense.csv'
Đang hợp nhất hai tệp thống kê...

Thành công! Đã tạo tệp tổng hợp đã định dạng tại: '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/sencount.csv'


In [ ]:
import pandas as pd
import os

def compare_sencount_files(file1_path, file2_path, key_column='predicate/type'):
    """
    So sánh hai tệp CSV giống sencount và in ra sự khác biệt.

    Args:
        file1_path (str): Đường dẫn đến tệp CSV thứ nhất.
        file2_path (str): Đường dẫn đến tệp CSV thứ hai.
        key_column (str): Tên cột định danh để so sánh (khóa chính).
    """
    # 1. Kiểm tra sự tồn tại của các tệp
    if not os.path.exists(file1_path):
        print(f"Lỗi: Không tìm thấy tệp '{file1_path}'")
        return
    if not os.path.exists(file2_path):
        print(f"Lỗi: Không tìm thấy tệp '{file2_path}'")
        return

    print(f"--- Bắt đầu so sánh hai tệp ---")
    print(f"Tệp 1: {os.path.basename(file1_path)}")
    print(f"Tệp 2: {os.path.basename(file2_path)}")

    # 2. Đọc hai tệp vào DataFrame
    df1 = pd.read_csv(file1_path)
    df2 = pd.read_csv(file2_path)

    # 3. Hợp nhất hai DataFrame để so sánh
    # 'outer' join để giữ lại tất cả các dòng từ cả hai tệp
    # 'indicator=True' để biết một dòng đến từ đâu (cả hai, chỉ tệp 1, hoặc chỉ tệp 2)
    merged_df = pd.merge(
        df1,
        df2,
        on=key_column,
        how='outer',
        suffixes=('_file1', '_file2'),
        indicator=True
    )

    # 4. Phân tích kết quả và in ra báo cáo

    # Tìm các dòng chỉ có trong tệp 1
    left_only = merged_df[merged_df['_merge'] == 'left_only']
    print(f"\n## 1. Các dòng chỉ có trong '{os.path.basename(file1_path)}': {len(left_only)} dòng")
    if not left_only.empty:
        print(left_only[[key_column]])

    # Tìm các dòng chỉ có trong tệp 2
    right_only = merged_df[merged_df['_merge'] == 'right_only']
    print(f"\n## 2. Các dòng chỉ có trong '{os.path.basename(file2_path)}': {len(right_only)} dòng")
    if not right_only.empty:
        print(right_only[[key_column]])

    # Tìm các dòng có ở cả 2 tệp nhưng giá trị khác nhau
    both_df = merged_df[merged_df['_merge'] == 'both'].copy()

    # Xác định các cột đếm để so sánh
    count_columns = [col for col in df1.columns if col != key_column]
    diffs = []

    for _, row in both_df.iterrows():
        is_different = False
        row_diff = {key_column: row[key_column]}
        for col in count_columns:
            val1 = row[f"{col}_file1"]
            val2 = row[f"{col}_file2"]
            if val1 != val2:
                is_different = True
                row_diff[f"{col}_file1"] = val1
                row_diff[f"{col}_file2"] = val2

        if is_different:
            diffs.append(row_diff)

    print(f"\n## 3. Các dòng có giá trị đếm khác nhau: {len(diffs)} dòng")
    if diffs:
        diff_df = pd.DataFrame(diffs)
        print(diff_df.to_string(index=False))

    print("\n--- So sánh hoàn tất ---")


# --- CẤU HÌNH ---
if __name__ == "__main__":
    # Đường dẫn cơ sở
    base_path = '/content/drive/MyDrive/Colab Notebooks/Do_An/'

    # === THAY ĐỔI TÊN HAI TỆP BẠN MUỐN SO SÁNH TẠI ĐÂY ===
    file1 = os.path.join('/content/drive/MyDrive/Colab Notebooks/Do_An', 'sencount.csv')
    file2 = os.path.join('/content/drive/MyDrive/Colab Notebooks/Do_An/DatasetForTraining/Corpus', 'sencount.csv')
    # =======================================================

    # Gọi hàm để thực hiện công việc
    # Giả sử bạn đã tạo 2 file sencount_v1.csv và sencount_v2.csv để thử
    # Đoạn code dưới đây tạo file mẫu để bạn có thể chạy ngay lập tức
    if not os.path.exists(file1) or not os.path.exists(file2):
        print("Tạo file mẫu để chạy thử...")
        sample_data1 = {
            'predicate/type': ['abolish.01', 'activate.01', 'add.01', 'be.01'],
            'egram': [75, 150, 200, 1000],
            'VE': [15, 20, 30, 0],
            'SUM': [90, 170, 230, 1000]
        }
        sample_data2 = {
            'predicate/type': ['abolish.01', 'activate.02', 'add.01', 'be.01'],
            'egram': [75, 100, 205, 1000],
            'VE': [15, 18, 30, 0],
            'SUM': [90, 118, 235, 1000]
        }
        pd.DataFrame(sample_data1).to_csv(file1, index=False)
        pd.DataFrame(sample_data2).to_csv(file2, index=False)
        print("Đã tạo file mẫu. Bắt đầu so sánh...")

    compare_sencount_files(file1, file2)

--- Bắt đầu so sánh hai tệp ---
Tệp 1: sencount.csv
Tệp 2: sencount.csv

## 1. Các dòng chỉ có trong 'sencount.csv': 6 dòng
   predicate/type
17     disrupt.01
23    initiate.01
27      mutate.01
31        skip.01
34  transcribe.01
40    truncate.01

## 2. Các dòng chỉ có trong 'sencount.csv': 7 dòng
   predicate/type
0      Disrupt.01
1     Initiate.01
2       Mutate.01
3         Skip.01
4   Transcribe.01
5     Truncate.01
41            NaN

## 3. Các dòng có giá trị đếm khác nhau: 2 dòng
predicate/type  VE_file1  VE_file2  per_file1  per_file2  SUM_file1  SUM_file2  rep_file1  rep_file2
     confer.01     241.0     243.0      151.0      153.0      486.0      488.0        NaN        NaN
proliferate.01      43.0      38.0       24.0       21.0      216.0      211.0       19.0       17.0

--- So sánh hoàn tất ---
